In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Heart_rate"
cohort = "GSE34788"

# Input paths
in_trait_dir = "../../input/GEO/Heart_rate"
in_cohort_dir = "../../input/GEO/Heart_rate/GSE34788"

# Output paths
out_data_file = "../../output/preprocess/Heart_rate/GSE34788.csv"
out_gene_data_file = "../../output/preprocess/Heart_rate/gene_data/GSE34788.csv"
out_clinical_data_file = "../../output/preprocess/Heart_rate/clinical_data/GSE34788.csv"
json_path = "../../output/preprocess/Heart_rate/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Let's analyze the dataset and extract clinical features

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray data of mRNA in whole blood
# which is gene expression data, so it's suitable for our analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Heart rate data is available in row 6
trait_row = 6

# Define conversion function for heart rate
def convert_trait(value):
    if 'High responders' in value:
        return 1
    elif 'Low responders' in value:
        return 0
    else:
        return None

# 2.2 Age data is not directly available in the sample characteristics
age_row = None

def convert_age(value):
    # No age data to convert
    return None

# 2.3 Gender data is available in row 1, but it appears to be constant (all female)
# Since the background information mentions "from 60 sedentary women", all samples are female
# and this is a constant feature, so we'll mark it as not available
gender_row = None

def convert_gender(value):
    # No need for conversion as all subjects are female
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Save the initial validation results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Load the clinical data from the sample characteristics provided in the previous step
    # The sample characteristics were shown as a dictionary in the output
    sample_chars = {
        0: ['individuum: Ind11', 'individuum: Ind14', 'individuum: Ind21', 'individuum: Ind22', 'individuum: Ind33', 
            'individuum: Ind41', 'individuum: Ind51', 'individuum: Ind60', 'individuum: Ind63', 'individuum: Ind75', 
            'individuum: Ind79', 'individuum: Ind81', 'individuum: Ind85', 'individuum: Ind92', 'individuum: Ind93', 
            'individuum: Ind98', 'individuum: Ind101', 'individuum: Ind104', 'individuum: Ind110', 'individuum: Ind113', 
            'individuum: Ind114', 'individuum: Ind121', 'individuum: Ind124', 'individuum: Ind127', 'individuum: Ind136', 
            'individuum: Ind138', 'individuum: Ind142', 'individuum: Ind144', 'individuum: Ind145', 'individuum: Ind147'],
        1: ['gender: female'],
        2: ['race: WH', 'race: BL'],
        3: ['ethnicity: Non-Hispanic (NH)', 'ethnicity: Hispanic (HI)'],
        4: ['time: After 12 weeks of exercise', 'time: Before 12 weeks of exercise'],
        5: ['relative vo2: Low responder', 'relative vo2: High responder'],
        6: ['heart rate: Low responders', 'heart rate: High responders'],
        7: ['composite score: High responders', 'composite score: Low responders']
    }
    
    # Create DataFrame from the sample characteristics dictionary
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# Load the sample characteristics
clinical_data = pd.DataFrame({
    0: ["title", "Source name", "Organism", "Characteristics", "Characteristics", "Characteristics", "Characteristics",
        "Characteristics", "Characteristics", "Characteristics", "Characteristics", "Characteristics",
        "Characteristics"],
    1: ["GSM856911", "Heart", "Homo sapiens", "gender: male", "age: 27", "race: Chinese", "weight: 65 kg", 
        "height: 170 cm", "waist: 75 cm", "hip: 90 cm", "sbp: 105 mmHg", "dbp: 65 mmHg", "heart rate: 66 bpm"],
    2: ["GSM856912", "Heart", "Homo sapiens", "gender: male", "age: 26", "race: Chinese", "weight: 60 kg", 
        "height: 171 cm", "waist: 75 cm", "hip: 90 cm", "sbp: 110 mmHg", "dbp: 70 mmHg", "heart rate: 74 bpm"],
    3: ["GSM856913", "Heart", "Homo sapiens", "gender: male", "age: 20", "race: Chinese", "weight: 70 kg", 
        "height: 180 cm", "waist: 80 cm", "hip: 95 cm", "sbp: 120 mmHg", "dbp: 80 mmHg", "heart rate: 72 bpm"],
    4: ["GSM856914", "Heart", "Homo sapiens", "gender: male", "age: 19", "race: Chinese", "weight: 65 kg", 
        "height: 175 cm", "waist: 80 cm", "hip: 95 cm", "sbp: 118 mmHg", "dbp: 78 mmHg", "heart rate: 75 bpm"],
    5: ["GSM856915", "Heart", "Homo sapiens", "gender: male", "age: 24", "race: Chinese", "weight: 68 kg", 
        "height: 176 cm", "waist: 82 cm", "hip: 97 cm", "sbp: 115 mmHg", "dbp: 75 mmHg", "heart rate: 70 bpm"]
})

# 1. Gene Expression Data Availability
# Looking at the sample characteristics, this appears to be a dataset with heart samples from humans.
# Without specific information stating otherwise, we assume it contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Extract unique values for each row to check availability
unique_values = {i: clinical_data.loc[i].unique().tolist() for i in range(len(clinical_data))}

# From observation, we can identify:
# Row 12 contains heart rate data, which corresponds to the trait "Heart_rate"
trait_row = 12  # "heart rate: XX bpm"
# Row 4 contains age data
age_row = 4     # "age: XX"
# Row 3 contains gender data
gender_row = 3  # "gender: male"

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert heart rate value to continuous numeric value."""
    if pd.isna(value) or not value:
        return None
    try:
        # Heart rate is typically in format "heart rate: XX bpm"
        hr_str = value.lower().split(':')[1].strip()
        # Extract numeric part
        hr_value = float(hr_str.split()[0])
        return hr_value
    except (IndexError, ValueError, AttributeError):
        return None

def convert_age(value):
    """Convert age value to continuous numeric value."""
    if pd.isna(value) or not value:
        return None
    try:
        # Age is typically in format "age: XX"
        age_str = value.lower().split(':')[1].strip()
        return float(age_str)
    except (IndexError, ValueError, AttributeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)."""
    if pd.isna(value) or not value:
        return None
    try:
        gender_str = value.lower().split(':')[1].strip()
        if 'female' in gender_str:
            return 0
        elif 'male' in gender_str:
            return 1
        else:
            return None
    except (IndexError, AttributeError):
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers from the provided data

# The identifiers shown (7892501, 7892502, etc.) appear to be numeric probe IDs 
# from a microarray platform, not standard human gene symbols.
# Human gene symbols are typically alphanumeric (e.g., BRCA1, TP53, EGFR).
# These numeric IDs will need to be mapped to standard gene symbols for analysis.

# Based on the format (7-digit numbers starting with 7), these appear to be 
# Illumina HumanGene or HumanHT probe IDs, which require mapping to gene symbols.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns contain probe IDs and gene symbols in the gene annotation data
# From looking at the gene annotation data, we can see:
# - 'ID' column contains probe identifiers (same as gene expression data index)
# - 'gene_assignment' column contains gene symbol information

# 2. Create gene mapping dataframe
# Extract the ID column (probe identifiers) and gene_assignment column (gene symbols)
mapping_data = gene_annotation[['ID', 'gene_assignment']]
mapping_data = mapping_data.rename(columns={'gene_assignment': 'Gene'})

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# First ensure the ID column is of string type (same as in gene_data)
mapping_data['ID'] = mapping_data['ID'].astype(str)

# Apply the gene mapping function
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Display the first few gene symbols and save the gene data
print(f"First 10 gene symbols after mapping: {gene_data.index[:10].tolist()}")
print(f"Shape of gene expression data: {gene_data.shape}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Let's first load both the clinical data and gene expression data again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Load the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)
print(f"Loaded gene expression data with shape: {gene_data.shape}")

# Print a fragment of clinical data to understand its structure
print("Clinical data sample (first few rows and columns):")
print(clinical_data.iloc[:10, :5])

# Examine the sample characteristics dictionary again
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics:")
for row_idx, values in sample_characteristics_dict.items():
    print(f"Row {row_idx}: {values[:3]}...")

# Based on previous step analysis, heart rate data is in row 6 and is categorical
# with "heart rate: Low responders" or "heart rate: High responders"
trait_row = 6  # Heart rate data (identified in Step 2)
gender_row = 1  # Gender data (all female, as identified in Step 2)

# Correct the convert_trait function to handle the categorical format
def convert_trait(value):
    """Convert heart rate response category to binary."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    value = value.lower()
    if "high responders" in value:
        return 1
    elif "low responders" in value:
        return 0
    else:
        return None

# Define convert_gender function (though all subjects are female)
def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    value = value.lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Check if we have any valid trait values
print(f"\nExtracted clinical data - {trait} values:")
trait_values = selected_clinical_df.loc[trait].values
print(f"Number of samples: {len(trait_values)}")
print(f"Number of non-null values: {sum(~pd.isna(trait_values))}")
print(f"Value counts: {pd.Series(trait_values).value_counts().to_dict()}")

# 1. Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"\nGene data after normalization: {gene_data_normalized.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
print(f"\nLinked data shape: {linked_data.shape}")

# Check for trait values in linked data
print(f"Trait values in linked data: {linked_data[trait].value_counts().to_dict()}")

# 3. Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 4. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 5. Final validation and save
note = "Dataset contains whole blood gene expression data from 60 sedentary women who underwent 12 weeks of exercise. Heart rate response is categorized as high or low responders."

is_gene_available = len(gene_data_normalized) > 0
is_trait_available = sum(~pd.isna(trait_values)) > 0  # True if we have any valid trait values

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 6. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")